# Lesson 1 solutions: Finding Higgs decays

<br><br><br><br><br>

## Getting data, building objects

In [1]:
import json

import numpy as np
import vector

dataset = json.load(open("../data/SMHiggsToZZTo4L.json"))

def to_vector(particle):
    return vector.obj(
        pt=particle["pt"],
        eta=particle["eta"],
        phi=particle["phi"],
        mass=particle["mass"],
    )

In [2]:
electrons_and_muons = []   # collectively known as "leptons"

event = dataset[96]   # a nice event with 3 electrons and 3 muons

for particle in event["electron"]:
    electrons_and_muons.append({
        "type": "electron",
        "charge": particle["charge"],
        "vector": to_vector(particle),
    })

for particle in event["muon"]:
    electrons_and_muons.append({
        "type": "muon",
        "charge": particle["charge"],
        "vector": to_vector(particle),
    })

In [3]:
z_candidates_step0 = []

for index_i, particle_i in enumerate(electrons_and_muons):
    for index_j, particle_j in enumerate(electrons_and_muons):
        if index_i < index_j:
            z_candidates_step0.append({
                "index": [index_i, index_j],
                "types": [particle_i["type"], particle_j["type"]],
                "charge": particle_i["charge"] + particle_j["charge"],
                "vector": particle_i["vector"] + particle_j["vector"],
            })

<br><br><br><br><br>

## Exercise part 1

Z bosons always decay into particles of opposite charge and identical flavor. Reduce the set of candidates by excluding ones with the wrong properties.

In [4]:
z_candidates_step1 = []

for candidate in z_candidates_step0:
    if candidate["charge"] == 0 and (
        candidate["types"] == ["electron", "electron"]
        or candidate["types"] == ["muon", "muon"]
    ):
        z_candidates_step1.append(candidate)

Print the masses of these Z boson candidates.

In [5]:
for candidate in z_candidates_step1:
    print(candidate["types"], candidate["vector"].mass)

['electron', 'electron'] 94.65200565609612
['electron', 'electron'] 3.417050436103103
['muon', 'muon'] 26.450245222365524
['muon', 'muon'] 3.2737370390906744


<br><br><br><br><br>

## Exercise part 2

The Higgs boson decays into two Z bosons. The only constraint here is that a lepton from one Z decay can't also be a lepton from the other Z decay.

In [6]:
higgs_candidates_step1 = []

for z_index1, z_candidate1 in enumerate(z_candidates_step1):
    for z_index2, z_candidate2 in enumerate(z_candidates_step1):
        if z_index1 < z_index2:
            lepton_i1, lepton_j1 = z_candidate1["index"]
            lepton_i2, lepton_j2 = z_candidate2["index"]
            if (
                (lepton_i1 != lepton_i2 and lepton_i1 != lepton_j2)
                and (lepton_j1 != lepton_i2 and lepton_j1 != lepton_j2)
            ):
                higgs_candidates_step1.append({
                    "z_candidates": [z_candidate1, z_candidate2],
                    "vector": z_candidate1["vector"] + z_candidate2["vector"],
                })

You could have also used

```python
if lepton_i1 not in (lepton_i2, lepton_j2) and lepton_j1 not in (lepton_i2, lepton_j2):
```

or several other, equivalent variants.

Print out the lepton indexes and candidate Higgs masses.

In [7]:
for higgs_candidate in higgs_candidates_step1:
    z_candidate1, z_candidate2 = higgs_candidate["z_candidates"]
    lepton_index1, lepton_index2 = z_candidate1["index"]
    lepton_index3, lepton_index4 = z_candidate2["index"]
    print(
        lepton_index1,
        lepton_index2,
        lepton_index3,
        lepton_index4,
        higgs_candidate["vector"].mass,
    )

0 2 3 4 129.03461596915852
0 2 3 5 118.8311777089631
1 2 3 4 56.10989169721264
1 2 3 5 12.750734071856018


Even though each candidate avoids double-counting within itself, the same combination of four indexes can be found among the candidates. We want only one of each.

Let's collect these Higgs candidates by unique sets of indexes. The `sorted` function sorts a list, and `tuple` makes it possible to use them as keys in a dict.

In [8]:
higgs_candidates_step2 = {}

for higgs_candidate in higgs_candidates_step1:
    z_candidate1, z_candidate2 = higgs_candidate["z_candidates"]
    lepton_index1, lepton_index2 = z_candidate1["index"]
    lepton_index3, lepton_index4 = z_candidate2["index"]

    combination = tuple(sorted([
        lepton_index1, lepton_index2, lepton_index3, lepton_index4
    ]))

    if combination not in higgs_candidates_step2:
        higgs_candidates_step2[combination] = []

    higgs_candidates_step2[combination].append(higgs_candidate)

This `higgs_candidates_step2` has deep structure:

  * Keys are sets combinations of lepton indexes, without regard for their original order.
  * Values are a list of decay trees.
    - Each element of that list has a candidate Higgs mass and two candidate Z masses.

In [9]:
for combination in higgs_candidates_step2:
    print(combination)
    for higgs_candidate in higgs_candidates_step2[combination]:
        z_candidate1, z_candidate2 = higgs_candidate["z_candidates"]
        print(
            "    Higgs:",
            higgs_candidate["vector"].mass,
            "Z:",
            z_candidate1["vector"].mass,
            z_candidate2["vector"].mass,
        )

(0, 2, 3, 4)
    Higgs: 129.03461596915852 Z: 94.65200565609612 26.450245222365524
(0, 2, 3, 5)
    Higgs: 118.8311777089631 Z: 94.65200565609612 3.2737370390906744
(1, 2, 3, 4)
    Higgs: 56.10989169721264 Z: 3.417050436103103 26.450245222365524
(1, 2, 3, 5)
    Higgs: 12.750734071856018 Z: 3.417050436103103 3.2737370390906744


<br><br><br><br><br>

## Exercise part 3

One of the selections that the 2012 Higgs discovery analysis applied was:

  * 12 GeV/$c^2$ < smallest Z mass < 120 GeV/$c^2$
  * 40 GeV/$c^2$ < largest Z mass < 120 GeV/$c^2$

because this is expected of real Higgs decays.

Apply the Z mass constraint to these Higgs candidates.

In [10]:
higgs_candidates_step3 = {}

for combination in higgs_candidates_step2:
    higgs_candidates_step3[combination] = []

    for higgs_candidate in higgs_candidates_step2[combination]:
        z_candidate1, z_candidate2 = higgs_candidate["z_candidates"]
        if z_candidate1["vector"].mass < z_candidate2["vector"].mass:
            smallest_z_mass = z_candidate1["vector"].mass
            largest_z_mass = z_candidate2["vector"].mass
        else:
            largest_z_mass = z_candidate1["vector"].mass
            smallest_z_mass = z_candidate2["vector"].mass

        if 12 < smallest_z_mass < 120 and 40 < largest_z_mass < 120:
            higgs_candidates_step3[combination].append(higgs_candidate)

You could have used

```python
smallest_z_mass, largest_z_mass = sorted([z_candidate1["vector"].mass, z_candidate2["vector"].mass])
```

or several other, equivalent, variants.

In the end,

In [11]:
for combination in higgs_candidates_step3:
    print(combination)
    for higgs_candidate in higgs_candidates_step3[combination]:
        z_candidate1, z_candidate2 = higgs_candidate["z_candidates"]
        print(
            "    Higgs:",
            higgs_candidate["vector"].mass,
            "Z:",
            z_candidate1["vector"].mass,
            z_candidate2["vector"].mass,
        )

(0, 2, 3, 4)
    Higgs: 129.03461596915852 Z: 94.65200565609612 26.450245222365524
(0, 2, 3, 5)
(1, 2, 3, 4)
(1, 2, 3, 5)


Only one of the combinations has a satisfactory Higgs candidate.

Its mass is about right, too (125 GeV).